In [9]:
import pandas as pd

In [10]:
data_csv = pd.read_csv('../data/raw/dataset.csv')
data_csv.head(2)

,subject,body,answer,type,queue,priority,language,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,Die Datenanalyse-Plattform brach unerwartet ab...,Ich werde Ihnen bei der Lösung des Problems he...,Incident,General Inquiry,low,de,Crash,Technical,Bug,Hardware,Resolution,Outage,Documentation,NaN
1,Customer Support Inquiry,Seeking information on digital strategies that...,We offer a variety of digital strategies and s...,Request,Customer Service,medium,en,Feedback,Sales,IT,Tech Support,NaN,NaN,NaN,NaN


### Ajouter la colone 'text'

In [11]:
data_csv['email'] = data_csv['subject'] +'/n'+ data_csv['body']

### Supprimer 'Subject' & 'body'

In [12]:
data_csv = data_csv.drop(columns=['subject','body'])

In [13]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 1. Préparation des ressources (à faire une seule fois)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# Chargement des listes de mots outils (stopwords)
STOPWORDS_DICT = {
    'en': set(stopwords.words('english')),
    'de': set(stopwords.words('german'))
}

def full_nlp_cleaning(text, lang):
    
    if pd.isna(text) or text == '':
        return []

    # --- ÉTAPE 1 : Normalisation (lowercase) ---
    text = text.lower()

    # --- ÉTAPE 2 : Suppression de la ponctuation ---
    text = re.sub(r'[^a-zàâçéèêëïîôùûüÿæœäöüß\s]', ' ', text)
    
    # Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()

    # --- ÉTAPE 3 : Tokenisation par langue ---
    lang_mapping = {'en': 'english', 'de': 'german'}
    nltk_lang = lang_mapping.get(lang, 'english')
    
    try:
        tokens = word_tokenize(text, language=nltk_lang)
    except Exception:
        # Fallback simple si la tokenisation NLTK échoue
        tokens = text.split()

    # --- ÉTAPE 4 : Suppression des stopwords ---
    stop_words = STOPWORDS_DICT.get(lang, STOPWORDS_DICT['en'])
    tokens = [t for t in tokens if t not in stop_words and len(t) > 1]

    return tokens

# --- APPLICATION SUR LE DATASET ---

data_csv['email_cleaned'] = data_csv.apply(lambda row: full_nlp_cleaning(row['email'], row['language']), axis=1)

# Visualiser le résultat
print(data_csv[['language', 'email', 'email_cleaned']].head())

# Sauvegarder le dataset nettoyé
data_csv.to_csv('dataset_cleaned.csv', index=False)

  language                                              email  \
0       de  Unvorhergesehener Absturz der Datenanalyse-Pla...   
1       en  Customer Support Inquiry/nSeeking information ...   
2       en  Data Analytics for Investment/nI am contacting...   
3       de  Krankenhaus-Dienstleistung-Problem/nEin Medien...   
4       en  Security/nDear Customer Support, I am reaching...   

                                       email_cleaned  
0  [unvorhergesehener, absturz, datenanalyse, pla...  
1  [customer, support, inquiry, nseeking, informa...  
2  [data, analytics, investment, ni, contacting, ...  
3  [krankenhaus, dienstleistung, problem, nein, m...  
4  [security, ndear, customer, support, reaching,...  
